In [1]:
from vietocr.tool.config import Cfg
from vietocr.tool.predictor import Predictor
import torch
from torch import nn

class OCREncoder(nn.Module):
    def __init__(self, model):
        
        super(OCREncoder, self).__init__()
        
        self.model = model

    def forward(self, img):
        src = self.model.cnn(img)
        memory = self.model.transformer.forward_encoder(src)
        return memory

config = Cfg.load_config_from_name('vgg_transformer')


config['weights']="./weights/vgg_transformer.pth"
config['device'] = 'cpu'

trainer = Predictor(config)

x = torch.randn(5, 3, 32, 160, requires_grad=True)

model = OCREncoder(trainer.model)
model.eval()
rs = model(x)
print(rs.shape)

/home/lhnguyen/.miniconda3/envs/kie-ner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lhnguyen/.miniconda3/envs/kie-ner/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lhnguyen/.miniconda3/envs/kie-ner/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([80, 5, 256])


In [2]:
dynamic_axes = {'input': {0: 'batch', 3: 'im_width'}, 'output': {0: 'feat_width', 1: 'batch'}}
torch.onnx.export(model,               # model being run
              x,                         # model input (or a tuple for multiple inputs)
              "transformer_encoder.onnx",   # where to save the model (can be a file or file-like object)
              export_params=True,        # store the trained parameter weights inside the model file
              opset_version=11,          # the ONNX version to export the model to
              do_constant_folding=True,  # whether to execute constant folding for optimization
              input_names = ['input'],   # the model's input names
              output_names = ['output'], # the model's output names
              dynamic_axes = dynamic_axes)

In [3]:
import onnxruntime as ort

sess = ort.InferenceSession('./transformer_encoder.onnx')

2023-03-13 21:27:03.939213596 [E:onnxruntime:, inference_session.cc:1500 operator()] Exception during initialization: /onnxruntime_src/onnxruntime/core/providers/cpu/tensor/transpose.h:48 onnxruntime::TransposeBase::TransposeBase(const onnxruntime::OpKernelInfo&) v >= 0 && static_cast<uint64_t>(v) <= std::numeric_limits<size_t>::max() was false. 



RuntimeException: [ONNXRuntimeError] : 6 : RUNTIME_EXCEPTION : Exception during initialization: /onnxruntime_src/onnxruntime/core/providers/cpu/tensor/transpose.h:48 onnxruntime::TransposeBase::TransposeBase(const onnxruntime::OpKernelInfo&) v >= 0 && static_cast<uint64_t>(v) <= std::numeric_limits<size_t>::max() was false. 
